In [ ]:
!pip install transformers

In [ ]:
!pip install adabelief-pytorch==0.2.0

In [ ]:
!pip install -U nbformat # 기설치시, update

In [ ]:
import os
import re
import numpy as np
from tqdm import tqdm
from glob import glob
import json
import requests
import tensorflow as tf
from transformers import BertModel, TFBertModel, TFRobertaModel, RobertaTokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel, AutoModelForSequenceClassification
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,KFold,StratifiedKFold
from adabelief_pytorch import AdaBelief
from transformers.optimization import get_cosine_schedule_with_warmup
from tqdm import tqdm, tqdm_notebook
import shutil
import gc
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import urllib.request

from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# 초기에 소스 파일이 위치하는 Colab 노트북 기본 파일 디렉토리로 이동
%cd /content/drive/MyDrive/KHUDA 금융/심화트랙

/content/drive/MyDrive/KHUDA 금융/심화트랙


In [ ]:
#random seed 고정
tf.random.set_seed(1234)
np.random.seed(1234)
BATCH_SIZE = 32
NUM_EPOCHS = 3

L_RATE = 1e-5
MAX_LEN = 128
max_grad_norm=1
log_interval=200
NUM_CORES = os.cpu_count()
device = torch.device("cuda:0")

DATA_IN_PATH = './데이터'
DATA_OUT_PATH = "./데이터"

In [ ]:
DATASET_URL = "https://raw.githubusercontent.com/ukairia777/finance_sentiment_corpus/main/finance_data.csv"
DATASET_NAME = "finance_data.csv"

In [ ]:
urllib.request.urlretrieve(DATASET_URL,
                           filename = DATASET_NAME
                           )

('finance_data.csv', <http.client.HTTPMessage at 0x794be59a7670>)

In [ ]:
dataset = pd.read_csv(DATASET_NAME)
dataset.head()

,labels,sentence,kor_sentence
0,neutral,"According to Gran, the company has no plans to...","Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로..."
1,neutral,Technopolis plans to develop in stages an area...,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
2,negative,The international electronic industry company ...,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,positive,With the new production plant the company woul...,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,positive,According to the company's updated strategy fo...,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."


In [ ]:
# 영어 원문 문장 제거
del dataset['sentence']
# 데이터셋 라벨데이터를 숫자로 치환, 라벨인코딩 사용해도 됨.
dataset['labels'] = dataset['labels'].replace(['neutral', 'positive', 'negative'],[0, 1, 2])

# 중복 데이터 제거
DATASET_PREP_FILE = './데이터/dataset_prep.csv'
dataset.drop_duplicates(subset = ['kor_sentence'], inplace = True)
dataset.to_csv(DATASET_PREP_FILE) # 구글 드라이브 내 data 폴더에 저장

<ipython-input-12-5fe54cf057e2>:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset['labels'] = dataset['labels'].replace(['neutral', 'positive', 'negative'],[0, 1, 2])


In [ ]:
#두가지 임베딩
#"klue/roberta-large"
#"monologg/kobigbird-bert-base"
#tokenizer = AutoTokenizer.from_pretrained("monologg/kobigbird-bert-base", cache_dir='bert_ckpt', do_lower_case=False)
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base", cache_dir='bert_ckpt', do_lower_case=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
dataset = pd.read_csv('./데이터/dataset_prep.csv')
dataset.drop('Unnamed: 0',axis=1,inplace=True)

In [ ]:
# 입출력 데이터 분리
X_data = dataset['kor_sentence']
y_data = dataset['labels']

In [ ]:
TEST_SIZE = 0.2 # Train: Test = 8 :2 분리
RANDOM_STATE = 42
# strtify = True 일 경우, 데이터 분리 이전의 라벨별 분포 고려
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data,
                                                    test_size = TEST_SIZE,
                                                    random_state = RANDOM_STATE,
                                                    stratify = y_data)

In [ ]:
print(f"훈련 입력 데이터 개수: {len(X_train)}")
print(f"테스트 입력 데이터 개수: {len(X_test)}")

훈련 입력 데이터 개수: 3861
테스트 입력 데이터 개수: 966


In [ ]:
# 훈련 데이터 라벨별 비율
y_train.value_counts(normalize = True)

,proportion
labels,
0,0.592852
1,0.282051
2,0.125097


In [ ]:
# 테스트 데이터 라벨별 비율
y_test.value_counts(normalize = True)

,proportion
labels,
0,0.592133
1,0.282609
2,0.125259


In [ ]:
# 데이터셋으로 변환해주기 위해 형식을 맞추도록 하겠습니다.
X=pd.concat([X_train,y_train],axis=1)
X.reset_index(drop=True,inplace=True)
X_test=X_test.reset_index()

In [ ]:
class TrainDataset(Dataset):
    def __init__(self, df):
        self.df_data = df
    def __getitem__(self, index):
        # get the sentence from the dataframe
        sentence = self.df_data.loc[index, 'kor_sentence']
        encoded_dict = tokenizer(
          text = sentence,
          add_special_tokens = True,
          max_length = MAX_LEN,
          pad_to_max_length = True,
          truncation=True,           # Pad & truncate all sentences.
          return_tensors="pt")

        padded_token_list = encoded_dict['input_ids'][0]
        token_type_id = encoded_dict['token_type_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        target = torch.tensor(self.df_data.loc[index, "labels"])
        sample = (padded_token_list, token_type_id , att_mask, target)
        return sample
    def __len__(self):
        return len(self.df_data)

In [ ]:
class TestDataset(Dataset):
    def __init__(self, df):
        self.df_data = df
    def __getitem__(self, index):
        # get the sentence from the dataframe
        sentence = self.df_data.loc[index, 'kor_sentence']
        encoded_dict = tokenizer(
          text = sentence,
          add_special_tokens = True,
          max_length = MAX_LEN,
          pad_to_max_length = True,
          truncation=True,           # Pad & truncate all sentences.
          return_tensors="pt")

        padded_token_list = encoded_dict['input_ids'][0]
        token_type_id = encoded_dict['token_type_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        sample = (padded_token_list, token_type_id , att_mask)
        return sample
    def __len__(self):
        return len(self.df_data)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
#AUto
train_data = TrainDataset(X)

test_data = TestDataset(X_test)

train_dataloader = torch.utils.data.DataLoader(train_data,
                                        batch_size=BATCH_SIZE,
                                        shuffle=True,
                                      num_workers=0)
test_dataloader = torch.utils.data.DataLoader(test_data,
                                        batch_size=BATCH_SIZE,
                                        shuffle=False,
                                      num_workers=0)
#두가지 모델
#"klue/roberta-base"
#kobigbird-bert-base
model = AutoModelForSequenceClassification.from_pretrained("klue/roberta-base",num_labels=3)



####미세조정
n=0
for name, child in model.named_children():
    if n==0:
      h=0
      for param in child.parameters():
        if h<=0: #이부분 숫자 조절로 fine-tuning => Roberta229: h=229
          param.requires_grad = False
        h+=1
    n+=1
#####
    # print(param)
model.to(device)
optimizer = AdaBelief(model.parameters(), lr=1e-5, eps=1e-16, betas=(0.9,0.999), weight_decouple = True, rectify = False)

warmup_ratio = 0.1
t_total = len(train_dataloader) * NUM_EPOCHS
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

for e in range(NUM_EPOCHS):
    train_acc = 0.0
    test_acc = 0.0
    best_acc =0.0
    model.train()
    torch.set_grad_enabled(True)
    for batch_id, (input_id,token_type_id,attention_mask,label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        input_id = input_id.long().to(device)
        token_type_id = token_type_id.long().to(device)
        attention_mask = attention_mask.long().to(device)
        label = label.to(device)
        outputs = model(input_ids=input_id, token_type_ids=token_type_id, attention_mask=attention_mask, labels=label)
        loss = outputs[0]
        out = outputs[1]
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        _, preds = torch.max(out, dim=1)
        train_acc += f1_score(preds.cpu(), label.cpu(),average='weighted')
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

preds = []
model.eval()
torch.set_grad_enabled(False)
for batch_id, (input_id,token_type_id,attention_mask) in enumerate(tqdm_notebook(test_dataloader)):
    input_id = input_id.long().to(device)
    token_type_id = token_type_id.long().to(device)
    attention_mask = attention_mask.long().to(device)
    outputs = model(input_ids=input_id, token_type_ids=token_type_id, attention_mask=attention_mask)
    out = outputs[0]
    for inp in out:
      preds.append(inp.detach().cpu().numpy())
Preds = np.array(preds)

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Please check your arguments if you have upgraded adabelief-pytorch from version 0.0.5.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  ---------
adabelief-pytorch=0.0.5  1e-08  False              False
>=0.1.0 (Current 0.2.0)  1e-16  True               True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended eps = 1e-8                                      Recommended eps = 1e-16
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.

Weight decoupling enabled in AdaBelief


<ipython-input-24-70fd2a362e84>:47: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (input_id,token_type_id,attention_mask,label) in enumerate(tqdm_notebook(train_dataloader)):


...

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2834: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2834: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input 

epoch 3 train acc 0.8971422573765364


<ipython-input-24-70fd2a362e84>:69: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (input_id,token_type_id,attention_mask) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/31 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2834: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
preds = []
model.eval()
torch.set_grad_enabled(False)
for batch_id, (input_id,token_type_id,attention_mask) in enumerate(tqdm_notebook(test_dataloader)):
    input_id = input_id.long().to(device)
    token_type_id = token_type_id.long().to(device)
    attention_mask = attention_mask.long().to(device)
    outputs = model(input_ids=input_id, token_type_ids=token_type_id, attention_mask=attention_mask)
    out = outputs[0]
    for inp in out:
      preds.append(inp.detach().cpu().numpy())
Preds = np.array(preds)

<ipython-input-25-2835772bb3d5>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (input_id,token_type_id,attention_mask) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/31 [00:00<?, ?it/s]

In [ ]:
# 모델이 예측한 라벨 도출
predicted_label = np.argmax(Preds, axis=1)


In [ ]:
predicted_label

array([1, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 1, 0, 1,
       2, 0, 1, 2, 0, 2, 2, 0, 1, 1, 1, 1, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 2, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 2, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1,
       1, 1, 2, 0, 0, 1, 0, 0, 2, 2, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 2, 2, 2, 0, 0, 0, 1, 0, 1, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0,
       2, 1, 0, 0, 2, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 2, 1, 1, 1, 0, 1, 2,
       0, 2, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 0, 0, 2,
       0, 2, 1, 0, 0, 0, 1, 0, 1, 2, 0, 0, 2, 1, 2, 1, 0, 2, 2, 0, 0, 1,
       0, 0, 2, 0, 1, 0, 1, 0, 1, 0, 0, 2, 1, 0, 0, 1, 0, 0, 0, 1, 2, 0,
       0, 0, 0, 1, 1, 0, 2, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 2, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 2, 2, 0, 0, 0, 1, 0, 0, 0, 0, 1, 2, 0, 1,

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, \
                            roc_auc_score, confusion_matrix, classification_report, \
                            matthews_corrcoef, cohen_kappa_score, log_loss

# Classification Report 저장
path2data='./데이터'
if not os.path.exists(path2data):
    os.mkdir(path2data)
CL_REPORT_FILE = "./데이터/cl_report.csv"

cl_report = classification_report(y_test, predicted_label, output_dict = True)
cl_report_df = pd.DataFrame(cl_report).transpose()
cl_report_df = cl_report_df.round(3)
cl_report_df.to_csv(CL_REPORT_FILE)
print(cl_report_df)

              precision  recall  f1-score  support
0                 0.895   0.879     0.887  572.000
1                 0.799   0.817     0.808  273.000
2                 0.824   0.851     0.837  121.000
accuracy          0.858   0.858     0.858    0.858
macro avg         0.839   0.849     0.844  966.000
weighted avg      0.859   0.858     0.859  966.000


In [ ]:
torch.save(model.state_dict(),'./klue_base_fold3_s.pth')

#여기서부터 데이터 적용. 학습은 위에서 끝

In [1]:

import torch.cuda.amp as amp
from transformers import XLMPreTrainedModel, XLMRobertaModel, XLMRobertaConfig, XLMRobertaTokenizer
from transformers import XLMRobertaForSequenceClassification, BertForSequenceClassification
from transformers import AutoTokenizer
from transformers import BertForSequenceClassification, DistilBertForSequenceClassification, XLNetForSequenceClassification,\
XLMRobertaForSequenceClassification, XLMForSequenceClassification, RobertaForSequenceClassification
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

In [2]:
tf.random.set_seed(1234)
np.random.seed(1234)

class args:
    # ---- factor ---- #
    debug=False
    amp = True
    gpu = '0'

    epochs=5
    batch_size=32
    weight_decay=1e-6
    n_fold=5
    fold=3 # [0, 1, 2, 3, 4] # 원래는 3

    exp_name = 'model_f'
    dir_ = f'./saved_models/'
    pt = 'mz_model'
    max_len = 128

    start_lr = 1e-5#1e-3,5e-5
    min_lr=1e-6
    # ---- Dataset ---- #

    # ---- Else ---- #
    num_workers=8
    seed=2021
    scheduler = None#'get_linear_schedule_with_warmup'


NameError: name 'tf' is not defined

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu
device = torch.device(f'cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base", cache_dir = 'bert_ckpt', do_lower_case = False)

In [ ]:
class TestDataset(Dataset):
    def __init__(self, df):
        self.df_data = df
    def __getitem__(self, index):
        # get the sentence from the dataframe
        sentence = self.df_data.loc[index, 'kor_sentence']
        encoded_dict = tokenizer(
          text = sentence,
          add_special_tokens = True,
          max_length = MAX_LEN,
          pad_to_max_length = True,
          truncation=True,           # Pad & truncate all sentences.
          return_tensors="pt")

        padded_token_list = encoded_dict['input_ids'][0]
        token_type_id = encoded_dict['token_type_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        sample = (padded_token_list, token_type_id , att_mask)
        return sample
    def __len__(self):
        return len(self.df_data)



In [ ]:
# 데이터 불러오기 및 전처리
df = pd.read_csv("")


NameError: name 'pd' is not defined

In [ ]:
test_data = TestDataset(df)

test_dataloader = torch.utils.data.DataLoader(test_data,
                                        batch_size=args.batch_size,
                                        shuffle=False,
                                      num_workers=0)

In [ ]:

def do_predict(net, valid_loader):

    val_loss = 0
    pred_lst = []
    logit=[]
    net.eval()
    for batch_id, (input_id,token_type_id,attention_mask) in enumerate(tqdm_notebook(valid_loader)):
        input_id = input_id.long().to(device)
        token_type_id = token_type_id.long().to(device)
        attention_mask = attention_mask.long().to(device)

        with torch.no_grad():
            if args.amp:
                with amp.autocast():
                    # output
                    output = net(input_ids=input_id, token_type_ids=token_type_id, attention_mask=attention_mask)[0]

            else:
                output = net(outputs = model(input_ids=input_id, token_type_ids=token_type_id, attention_mask=attention_mask))

            pred_lst.extend(output.argmax(dim=1).tolist())
            logit.extend(output.tolist())

    return pred_lst,logit

def run_predict(model_path,test_dataloader):


    print('set testloader')
    ## net ----------------------------------------
    scaler = amp.GradScaler()

    net = RobertaForSequenceClassification.from_pretrained('klue/roberta-base', num_labels = 3)


    net.to(device)

    if len(args.gpu)>1:
        net = nn.DataParallel(net)

    f = torch.load(model_path)
    net.load_state_dict(f, strict=True)  # True
    print('load saved models')
    # ------------------------
    # validation
    preds, logit = do_predict(net, test_dataloader) #outputs

    print('complete predict')

    return preds, np.array(logit)

In [ ]:
preds1, logit1 = run_predict('./klue_base_fold3_s.pth',test_dataloader)

set testloader


<ipython-input-63-08a68ad77adc>:31: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-63-08a68ad77adc>:41: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` wil

load saved models


<ipython-input-63-08a68ad77adc>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (input_id,token_type_id,attention_mask) in enumerate(tqdm_notebook(valid_loader)):


  0%|          | 0/93 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2834: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
<ipython-input-63-08a68ad77adc>:14: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


complete predict


In [ ]:
print(type(logit1))  # 타입 확인
print(logit1.dtype)  # 데이터 타입
print(logit1.shape)  # 배열의 차원 (행, 열)
print(logit1.size)   # 총 원소 수
print(logit1.ndim)   # 차원 수

<class 'numpy.ndarray'>
float64
(2967, 3)
8901
2


In [ ]:
print(logit1)

[[-0.57861328  2.734375   -1.51660156]
 [-0.32470703  2.39257812 -1.59082031]
 [ 2.17578125 -0.28491211 -1.96386719]
 ...
 [ 1.43945312  1.29492188 -2.41015625]
 [ 2.61523438 -0.74414062 -2.00195312]
 [-0.64257812 -1.79296875  2.21679688]]


In [ ]:
import numpy as np

# logit1: 각 클래스(긍정, 중립, 부정)에 대한 정규화되지 않은 점수 리스트
# 예제 logit1 (실제 데이터로 교체하세요)

# 소프트맥스 함수 정의
def softmax(logits):
    exp_values = np.exp(logits - np.max(logits, axis=-1, keepdims=True))  # 안정성을 위해 max 값 빼기
    probabilities = exp_values / np.sum(exp_values, axis=-1, keepdims=True)
    return probabilities

# logit1을 소프트맥스 변환
new_logit1 = [softmax(logit) for logit in logit1]

# 결과 확인
print("Softmax 변환 후 확률:")
for i, probs in enumerate(new_logit1):
    print(f"Logit {i + 1}: {probs}")


Softmax 변환 후 확률:
Logit 1: [0.03465184 0.95178493 0.01356323]
Logit 2: [0.06089729 0.92193422 0.01716849]
Logit 3: [0.90801435 0.07752239 0.01446326]
Logit 4: [0.01957411 0.96738672 0.01303917]
Logit 5: [0.90801435 0.07752239 0.01446326]
Logit 6: [0.02266249 0.96504617 0.01229134]
Logit 7: [0.02213297 0.964377   0.01349003]
Logit 8: [0.05552068 0.92789423 0.01658508]
Logit 9: [0.21469549 0.77428153 0.01102298]
Logit 10: [0.45602827 0.53159139 0.01238034]
Logit 11: [0.08083315 0.0161716  0.90299525]
Logit 12: [0.06554225 0.92431903 0.01013872]
Logit 13: [0.18046593 0.80681999 0.01271408]
Logit 14: [0.02259963 0.96566438 0.01173599]
Logit 15: [0.45476933 0.53480366 0.01042701]
Logit 16: [0.87501935 0.1144907  0.01048995]
Logit 17: [0.25189618 0.72995426 0.01814955]
Logit 18: [0.15802543 0.01914435 0.82283022]
Logit 19: [0.02507126 0.96263218 0.01229656]
Logit 20: [0.31064741 0.67752314 0.01182945]
Logit 21: [0.74418191 0.24588492 0.00993317]
Logit 22: [0.76987319 0.21832219 0.01180463]
Lo

In [ ]:
import pandas as pd

# 1. 기존 데이터프레임(df)와 감성 분석 결과(preds1, logit1)를 준비했다고 가정합니다.
# df['news_title']: 기사 제목이 담긴 데이터프레임의 칼럼
# preds1: 감성 분석 결과 (라벨, 예: 'Positive', 'Neutral', 'Negative')
# logit1: 각 감성에 대한 확률 (2D 리스트 또는 배열 형태)

# 예제 데이터 (실제 데이터로 대체하세요)
# df = pd.read_csv("input.csv")
# preds1 = ['Positive', 'Negative', 'Neutral']  # 감성 라벨
# logit1 = [[0.8, 0.1, 0.1], [0.2, 0.3, 0.5], [0.4, 0.4, 0.2]]  # 확률

# 2. 새로운 데이터프레임 생성
output_df = pd.DataFrame({
    'Article Title': df['kor_sentence'],  # 기사 제목
    'Date' : df["Date"],
    'Sentiment Label': preds1,  # 감성 라벨
    'Neutral Probability': [x[0] for x in new_logit1],  # 긍정 확률
    'Positive Probability': [x[1] for x in new_logit1],   # 중립 확률
    'Negative Probability': [x[2] for x in new_logit1]  # 부정 확률
})

# 3. 새로운 파일로 저장
output_file = "삼전_sentiment_results.csv"  # 출력 파일 이름
output_df.to_csv(output_file, index=False, encoding='utf-8-sig')  # UTF-8로 저장 (한글 호환)

print(f"결과가 {output_file}에 저장되었습니다!")


결과가 삼전_sentiment_results.csv에 저장되었습니다!


numpy.ndarray